In [1]:
import os
import glob 
import re

from datetime import datetime
import earthpy as et
import geojson
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pytz


In [2]:
pwd

'/Users/robynmarowitz/projects/tempo-site/data/textile-source'

In [3]:
os.chdir("../monthly")
# print(glob.glob('*'))

In [4]:
file_list = glob.glob("*.geojson")
file_list

['HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_022024_15Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_062024_01Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_082023_11Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_082024_18Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_052024_18Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_072024_13Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_112023_19Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_012024_22Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_042024_16Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_082023_23Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_102023_17Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_072024_21Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_052024_11Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_082024_11Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_082023_18Z_V3.geojson',
 'HAQ_TEMPO_NO2_CONUS_QA75_L3_Monthly_032024_20Z_V3.geojson',
 'HAQ_TE

In [5]:
def extract_date_from_filename(filename):
    # Use regular expression to capture the date between 'Monthly_' and '_V3'
    match = re.search(r'Monthly_(\d{6})', filename)
    if match:
        return match.group(1)
    else:
        return None  # In case the date is not found

In [6]:
def extract_time_from_filename(filename):
    match = re.search(r'(\d{2}Z)', filename)
    if match:
        hour_int = match.group(1)[:2]
        return hour_int
    else:
        return None  # In case the time is not found

In [7]:
def convert_to_mountain_time(utc_hour):
    if utc_hour is not None:
        # Create a datetime object in UTC
        utc_time = datetime.strptime(f'{utc_hour}:00:00', '%H:%M:%S')
        utc_time = pytz.utc.localize(utc_time)  # Localize to UTC
        
        # Convert to Mountain Time (Colorado)
        mountain_time = utc_time.astimezone(pytz.timezone('America/Denver'))
        
        # Return the hour in Mountain Time
        return mountain_time.hour
    else:
        return None  # Return None if the time is invalid or missing

In [ ]:
gdfs = []
for file in file_list:
    # print(file)
    gdf = gpd.read_file(file)
    # Extract the date from the filename
    date_str = extract_date_from_filename(file)
    time_int = extract_time_from_filename(file)
    gdf['date'] = pd.to_datetime(date_str, format='%m%Y')
    gdf['hour_utc'] = time_int
    gdfs.append(gdf)

gdfs_cleaned = [gdf.dropna(axis=1, how='all') for gdf in gdfs]
gdfs_cleaned[0]

In [ ]:
monthly_gdf = pd.concat(gdfs_cleaned, ignore_index=True)
monthly_gdf

In [ ]:
monthly_gdf['hour_mountain'] = monthly_gdf['hour_utc'].apply(convert_to_mountain_time)
monthly_gdf

In [ ]:
# I only want fips 8013 and and 8031
fips_to_keep = ["8013", "8031"] 
monthly_gdf["FIPS_new"] = monthly_gdf["FIPS_new"].astype(str)
filtered_gdf = monthly_gdf[monthly_gdf["FIPS_new"].isin(fips_to_keep)]

In [ ]:
census_gdf = gpd.read_file('../preprocess/Colorado_Census_Tract_Boundaries.geojson')
# census_gdf

In [ ]:
df = pd.read_csv('../preprocess/state_and_county_fips_master.csv')
# df

In [ ]:
co_df = df[df['state']=='CO'] # Filter to only Colorado
# Create new column with County FIPS from tract fips
census_gdf['FIPS_new'] = census_gdf['FIPS'].str[:5].str.lstrip('0').astype(int)
census_gdf

In [ ]:
filtered_gdf = filtered_gdf.to_crs(census_gdf.crs)
census_gdf['FIPS_new'] = census_gdf['FIPS_new'].astype('int64')
filtered_gdf['FIPS_new'] = filtered_gdf['FIPS_new'].astype('int64')
filtered_gdf

In [ ]:
joined_gdf_1 = gpd.sjoin(filtered_gdf, census_gdf, how='inner', predicate='intersects')  # or use 'within', 'contains', etc.
# joined_gdf_1

In [ ]:
joined_gdf = census_gdf.merge(filtered_gdf, on='FIPS_new', how='inner')  # Change 'inner' to 'left', 'right', or 'outer' if needed

# joined_gdf.head()

In [ ]:
bins = [0, 1, 2, 3, 4, 5, 6, float('inf')]
bin_labels = ['0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6+']

filtered_gdf['field_avg_bins'] = pd.cut(filtered_gdf['field_avg'], bins=bins, labels=bin_labels, right=False)
monthly_gdf['field_avg_bins'] = pd.cut(monthly_gdf['field_avg'], bins=bins, labels=bin_labels, right=False)


In [ ]:
filtered_gdf_bin_counts = filtered_gdf['field_avg_bins'].value_counts().sort_index()
monthly_gdf_bin_counts = monthly_gdf['field_avg_bins'].value_counts().sort_index()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot for filtered_gdf
filtered_gdf_bin_counts.plot(kind='bar', ax=axes[0], color='skyblue')
axes[0].set_title('Filtered GDF field_avg Distribution')
axes[0].set_xlabel('field_avg Ranges')
axes[0].set_ylabel('Count')

# Plot for monthly_gdf
monthly_gdf_bin_counts.plot(kind='bar', ax=axes[1], color='salmon')
axes[1].set_title('Monthly GDF field_avg Distribution')
axes[1].set_xlabel('field_avg Ranges')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
color_map = {
    '0-1': 'royalblue',
    '1-2': 'teal',
    '2-3': 'seafoam',  # may need to specify a hex code for 'seafoam', e.g., '#9FE2BF'
    '3-4': 'yellow',
    '4-5': 'orange',
    '5-6': 'salmon',
    '6+': 'maroon'  # Highest value
}


### hex colors for the range
Dark Blue (0-1): #00008B  
Teal (1-2): #008080  
Seafoam (2-3): #2E8B57  
Yellow (3-4): #FFFF00  
Orange (4-5): #FFA500  
Salmon (5-6): #FA8072  
Maroon (6+): #800000

In [ ]:
filtered_gdf['color'] = filtered_gdf['field_avg_bins'].map(color_map)
filtered_gdf

In [ ]:
# Exclude rows where 'field_avg' is NaN
filtered_gdf = filtered_gdf.dropna(subset=['field_avg'])

# Now split the filtered DataFrame into FIPS_8013 and FIPS_8031
gdf_fips_8013 = filtered_gdf[filtered_gdf['FIPS_new'] == 8013]
gdf_fips_8031 = filtered_gdf[filtered_gdf['FIPS_new'] == 8031]


In [ ]:
# Drop the geometry column and export to CSV
gdf_fips_8013.drop(columns='geometry').to_csv('fips_8013.csv', index=False)
gdf_fips_8031.drop(columns='geometry').to_csv('fips_8031.csv', index=False)


In [ ]:
# get specifically "08013012300" - boulder
target_fips = '08013012300'

monthly_gdf[monthly_gdf['FIPS'].isin([target_fips])]
gdf_fips_8013_specific = filtered_gdf[filtered_gdf['FIPS'] == target_fips]
gdf_fips_8013_specific.drop(columns='geometry').to_csv('fips_08013012300.csv', index=False)
